# Simplicial Complex Analysis

<span style='color:Red'>TODO: Get more robust numbers working for when to display summary information for shapes as a whole and the largest shape.</span> <br>
<span style='color:Red'> Currently using the largest number of nodes in each shape. </span> <br> 
Written by Frederick Miller, Casey McKean, and Wako Bungula
The kepler mapper object gives an output that is not easily navigatible. To resolve this, we wish to create shapes that are easier to navigate and understand, and reveal the data inside of them. <br>
We generate all the shapes in the simplicial complex, condense 1-simplices where possible, and obtain summary statistics on the shapes and the nodes within the shapes.

In [11]:
import numpy as np
import pandas as pd 
import queue
import json
pd.set_option('display.max_rows', None)
print("Imports Done")

Imports Done


# File paths and `.json` import

From the `kmapper_demo` file, I added one extra code block to place the results in a `.json` file, which is a way to store dictionaries in long term storage. The code below only needs to have the file paths changed, and then it will read the simplicial complices generated from kepler mapper. <br>
Here, we also import the actual data set, with data interpolated for the specific pool. <br>
Lastly, a list of the 11 continuous variables (the interpolated versions) is created.

In [12]:
jsonFilePath = r"C:\Users\forre\Desktop\REU\TDA\Data\TDAOutputs\TDA1PCA10Cubes45perc.json"
jsonFile = open(jsonFilePath, "r")
jsonData = json.load(jsonFile) 
jsonFile.close()

dataFilePath = r"C:\Users\forre\Desktop\REU\TDA\Data\interpolatedPool26.csv"
df = pd.read_csv(dataFilePath)

variables = ['PREDICTED_TN',
             'PREDICTED_TP',
             'PREDICTED_TEMP',
             'PREDICTED_DO',
             'PREDICTED_TURB',
             'PREDICTED_COND',
             'PREDICTED_VEL',
             'PREDICTED_SS',
             'PREDICTED_WDP',
             'PREDICTED_CHLcal',
             'PREDICTED_SECCHI']
print("Json file imported")

print(len(jsonData.keys()))

Json file imported
25


# Functions
See the `docstring`'s for what each function does and how it works.

In [13]:
def getSubdf(scomplex, shape, df):
    """
    Returns the part of the data frame from the particular shape in the simplicial complex.
    params:
    scomplex: the entire simplicial complex
    shape: the particular shape being inspected (within the simplicial complex)
    df: the entire data frame
    
    Description:
    1. Get all the nodes from the particular simplicial complex. 
    2. Generate the indices we care about from the particular shape. To do this, we read each node and append it's 
    indices to a list. Then, we convert the list to a set and then back to a list to eliminate duplicates.
    3. Return the dataframe with only those indices.
    """
    nodes = scomplex.get('nodes')
    indices = []
    for node in shape:
        indices.append(nodes.get(node))
    indices = list(set([item for sublist in indices for item in sublist]))
    subdf = df.loc[indices]
    return subdf

def shapeDataSummary(scomplex, shape, df, variables, verbose = False):
    """
    Generates summary statistics of the given variables for a given shape in the simplicial.
    params:
    scomplex: the entire simplicial complex
    shape: the particular shape being inspected (within the simplicial complex) at this function call.
    df: the entire dataframe
    variables: the variables of interest
    verbose: Determines if the function will print out extra information. False by default
    
    Description:
    1. Create an empty result dataframe to store the summary statistics.
    2. Get the sub dataframe (see getSubdf) for the particular shape
    3. For each variable we are analzying, generate summary statistics from the sub dataframe and place them
    inside the result dataframe.
    4. Return the result dataframe
    
    NOTE: this only creates summaries for one particular shape. In executing this method, it is done for each shape 
    outside of the function.
    
    """
    result = pd.DataFrame()
    if verbose == True:
        print("Obtaining sub dataframe for: ", shape)
        print("The number of nodes in this shape is: ", len(shape))
    subdf = getSubdf(scomplex, shape, df)
    if verbose == True:
        print("The number of datapoints in this shape is: ", subdf.shape[0])
    for var in variables:
        result[var] = subdf[var].describe()
    return result
    
    

def adjacent(v, scomplex):
    """
    Determines the nodes adjacent to a given vertex
    
    params:
    v: vertex
    scomlex: the entire simplicial complex
    
    Description:
    Determines the nodes that are adjacent to a given vertex.
    """
    
    simplices = scomplex.get('simplices')
    edges = [item for item in simplices if len(item) == 2]
    result = []
    for edge in edges:
        if v in edge:
            for item in edge:
                if item != v:
                    result.append(item)
    return result

def bfs(node, scomplex):
    """
    Conducts a breadth first search to obtain the entire shape from a given node
    params:
    node: the start node
    scomplex: the entire simplicial complex
    
    Description:
    Preforms a breadth first search to obtain the entire shape for a given start node.
    """
    Q = queue.Queue()
    result = []
    result.append(node)
    Q.put(node)
    while not Q.empty():
        v = Q.get()
        adjacentEdges = adjacent(v, scomplex)
        for edge in adjacentEdges:
            if edge not in result:
                result.append(edge)
                Q.put(edge)
    return result


        
    
def getShapes(scomplex):
    """
    Gets all of the shapes from a given simplicial complex.
    
    params:
    scomplex: the entire simplicial complex
    
    Description:
    1. Obtain all the nodes for the entire complex
    2. For each node, preform a breadth first search to obtain everything in that particular shape. 
    If this entire shape has not already been discovered, add it to the set of results. 
    The result item is a set as the order of the shapes does not matter. The resulting shape is a frozenset
    which means items cannot be added or removed once created, and is needed to allow the set object to have other sets within it.
    3. Convert each shape to a list and the result to a list for easier navigation outside of the function.
    4. Return the result
    
    """
    
    nodes = list(scomplex.get('nodes').keys())
    result = set()
    for node in nodes: # currently does more computations than necessary due to going through every node without considering it is already in a shape
        bfsResult = frozenset(bfs(node, scomplex))
        result.add(bfsResult)
    result = [list(x) for x in result]
    # Sort the list depending on what is decided: nodes or indices. Currently doing it by number of nodes
    result.sort(key = len, reverse = True)
    
    
    return result

def nodeDataSummary(node, scomplex, variables,df):
    """
    Returns a data summary of a particular node
    params:
    node: node in question
    scomplex: The entire simplicial complex
    variables: The variables to obtain summaries
    df: the entire dataframe 
    
    description:
    1. Creates a result dataframe
    2. Get all the indices from the node from the simplicial complex
    3. Generate summaries for each variable
    4. Return the result
    """
    result = pd.DataFrame()
    indices = scomplex.get('nodes').get(node)
    subdf = df.loc[indices]
    for var in variables:
        result[var] = subdf[var].describe()
    return result
    
    
def condenseShape(shape, scomplex):
    """
    
    params:
    shape: a shape of two nodes. must be 2
    scomplex: the entire simplicial complex
    
    description:
    gets the two nodes a and b
    gets the indices for a and b (what is inside the nodes)
    if a \subseteq b, return b
    elif b \subseteq a, return a 
    else return shape 
    
    """
    nodes = scomplex.get('nodes')
    a = shape[0]
    b = shape[1]
    aIndices = set(nodes.get(a))
    bIndices = set(nodes.get(b))
    
    if aIndices.issubset(bIndices):
        return b
    elif bIndices.issubset(aIndices):
        return a
    else:
        return shape

def clean_getShapes(scomplex):
    """
    Condenses 1-simplices down to 0-simplices when each node 
    is a subset of the other 
    
    params:
    scomplex: the entire simplicial complex
    
    Description:
    1. Get all the shapes from the original getShapes function
    2. For shapes that of length 2, if one is a subset of the other, return the larger of the two
        Otherwise, do nothing
    3. return the clean Shapes list 
    
    """
    shapes = getShapes(scomplex)
    cleanShapes = []
    for shape in shapes:
        if len(shape) == 2:
            shape = condenseShape(shape, scomplex)
            cleanShapes.append([shape])
        else:
            cleanShapes.append(shape)
    return cleanShapes
print("Functions loaded")

Functions loaded


# Generating Summary Statistics on the entire simplicial complex
For each `mapper` output from `kepler-mapper`, we can generate the summary statistics for each of the continuous variables. This is done by first obtaining a list of the keys from the `.json` file, and then iterating through each complex, generating the shape and obtaining data summaries on each shape.

In [14]:
allComplices = list(jsonData.keys())
for key in allComplices[0:2]: # remove indices here to get all the strata for all the time periods
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    shapes = clean_getShapes(scomplex)
    for shape in shapes:
        summaries = shapeDataSummary(scomplex, shape, df, variables, verbose = False)
        if summaries.loc['count'][0] > 5 and len(shape)  > 2: # at least 6 datapoints and 3 nodes to see info
            print("The shape is: ",shape)
            print("The number of nodes in the shape is: ", len(shape))
            display(summaries) # Uncomment to see summaries

Current Simplical Complex:  ['Stratum 1 SUMMER 93-00: ']
The shape is:  ['cube7_cluster1', 'cube6_cluster0', 'cube3_cluster0', 'cube5_cluster0', 'cube4_cluster0', 'cube5_cluster1', 'cube5_cluster5', 'cube7_cluster0', 'cube2_cluster0', 'cube5_cluster11', 'cube1_cluster0', 'cube2_cluster6']
The number of nodes in the shape is:  12


,PREDICTED_TN,PREDICTED_TP,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_COND,PREDICTED_VEL,PREDICTED_SS,PREDICTED_WDP,PREDICTED_CHLcal,PREDICTED_SECCHI
count,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000
mean,3.321725,0.218013,14.083784,9.522973,54.986486,475.972973,0.330604,60.462093,3.074189,55.155801,26.432432
std,0.941895,0.073410,2.574055,2.204637,30.887553,55.188338,0.332414,34.098436,2.673006,34.438192,8.146066
min,1.730722,0.112000,6.500000,3.600000,21.000000,347.000000,0.000000,16.400000,0.400000,10.457550,9.000000
25%,2.475142,0.170719,13.000000,8.325000,37.000000,445.250000,0.000000,38.506935,1.105000,23.575204,23.000000
50%,3.334431,0.200094,14.000000,9.800000,43.000000,465.500000,0.310000,51.442821,1.800000,54.557334,27.000000
75%,4.061680,0.238260,15.200000,11.375000,60.000000,515.000000,0.565000,71.879000,4.525000,84.450847,30.000000
max,5.081000,0.429201,21.500000,13.600000,150.000000,587.000000,1.580000,187.900000,10.400000,128.955279,50.000000


Current Simplical Complex:  ['Stratum 2 SUMMER 93-00: ']
The shape is:  ['cube1_cluster1', 'cube4_cluster2', 'cube6_cluster0', 'cube4_cluster1', 'cube3_cluster0', 'cube1_cluster4', 'cube5_cluster4', 'cube5_cluster1', 'cube7_cluster0', 'cube2_cluster0', 'cube2_cluster8', 'cube5_cluster2']
The number of nodes in the shape is:  12


,PREDICTED_TN,PREDICTED_TP,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_COND,PREDICTED_VEL,PREDICTED_SS,PREDICTED_WDP,PREDICTED_CHLcal,PREDICTED_SECCHI
count,200.000000,200.000000,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000,200.000000
mean,3.320844,0.233468,16.36550,9.161500,63.647786,437.660000,0.300183,71.453582,2.978300,51.520813,24.945000
std,1.216655,0.103036,4.22948,2.479031,46.724682,83.955865,0.347283,53.672637,2.509813,36.177092,9.094809
min,1.358000,0.033000,6.50000,3.600000,15.000000,227.000000,0.000000,16.400000,0.400000,9.239100,5.000000
25%,2.332597,0.171971,13.90000,7.600000,37.000000,374.000000,0.000000,38.687777,1.100000,25.189010,19.000000
50%,3.227672,0.225356,15.00000,8.600000,51.000000,450.000000,0.133885,54.025000,1.900000,40.114605,25.000000
75%,4.053040,0.282344,17.62500,10.625000,70.250000,490.750000,0.552500,87.322971,4.300000,63.834940,30.000000
max,9.568764,0.972000,28.00000,19.600000,360.000000,697.000000,1.580000,307.700000,11.900000,183.861190,50.000000


# Analyzing the largest structure
Largest = Node count of the shape. The largest structure is likely to be the dominant feature of the stratum during this particular time period. As such, it is important to analyze the nodes within it. To do this, we generate all the shapes, and since the shapes are returned in descending order of the number of nodes per shape, we pull the first shape. From here, we can preform an analysis on each one.

In [6]:
allComplices = list(jsonData.keys())
for key in allComplices[0:3]: # remove the indices here to get all the strata for all the time periods
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    largestShape = clean_getShapes(scomplex)[0]
    nodes = scomplex.get('nodes')
    print("Largest shape is: ", largestShape)
    print("Number of nodes is: ", len(largestShape))
    for node in largestShape:
        summary = nodeDataSummary(node, scomplex,variables,df)
        if summary.loc['count'][0] > 5: # 5 is chosen arbitraily
            print("Information for: ", node)
            display(summary)

Current Simplical Complex:  ['Stratum 1 SUMMER 93-00: ']
Largest shape is:  ['cube7_cluster1', 'cube6_cluster0', 'cube3_cluster0', 'cube5_cluster0', 'cube4_cluster0', 'cube5_cluster1', 'cube5_cluster5', 'cube7_cluster0', 'cube2_cluster0', 'cube5_cluster11', 'cube1_cluster0', 'cube2_cluster6']
Number of nodes is:  12
Information for:  cube6_cluster0


,PREDICTED_TN,PREDICTED_TP,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_COND,PREDICTED_VEL,PREDICTED_SS,PREDICTED_WDP,PREDICTED_CHLcal,PREDICTED_SECCHI
count,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000
mean,3.104918,0.218218,14.715152,8.915152,60.575758,468.393939,0.362554,63.492205,3.023333,45.399381,25.030303
std,0.933327,0.089223,2.329716,1.837308,36.170629,55.167325,0.356081,36.833492,2.688964,33.196787,9.524589
min,1.730722,0.112000,6.500000,4.300000,21.000000,353.000000,0.000000,16.400000,0.500000,10.457550,9.000000
25%,2.388667,0.158000,14.000000,8.000000,38.000000,433.000000,0.000000,40.049999,1.100000,18.908110,20.000000
50%,3.023000,0.180000,14.900000,9.000000,46.000000,460.000000,0.343460,51.585642,1.800000,32.315664,27.000000
75%,3.661000,0.274000,15.900000,10.200000,71.000000,511.000000,0.570000,78.600000,4.300000,63.726140,30.000000
max,5.081000,0.429201,18.500000,12.200000,150.000000,577.000000,1.580000,187.900000,10.400000,128.955279,50.000000


Information for:  cube3_cluster0


,PREDICTED_TN,PREDICTED_TP,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_COND,PREDICTED_VEL,PREDICTED_SS,PREDICTED_WDP,PREDICTED_CHLcal,PREDICTED_SECCHI
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,3.518011,0.206031,14.256250,10.606250,43.687500,491.56250,0.326901,55.139887,3.325000,65.554711,28.875000
std,0.955304,0.049266,2.283263,1.339387,14.938624,42.36031,0.285486,23.954470,2.813894,33.395974,4.716991
min,2.005000,0.113500,10.800000,8.100000,24.000000,438.00000,0.000000,25.050000,0.500000,13.880215,20.000000
25%,2.705777,0.176484,13.150000,9.500000,37.000000,466.25000,0.000000,38.375000,1.350000,39.898940,26.000000
50%,4.027500,0.204094,13.700000,10.800000,41.000000,480.50000,0.435000,50.534611,2.250000,67.599420,29.000000
75%,4.108000,0.217750,15.025000,11.650000,45.000000,498.75000,0.515000,67.287500,4.400000,87.052860,31.500000
max,5.079000,0.323000,21.000000,12.200000,90.000000,587.00000,0.770000,108.600000,10.100000,121.694780,38.000000


Information for:  cube5_cluster0


...

Information for:  cube2_cluster8


,PREDICTED_TN,PREDICTED_TP,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_COND,PREDICTED_VEL,PREDICTED_SS,PREDICTED_WDP,PREDICTED_CHLcal,PREDICTED_SECCHI
count,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000,15.000000
mean,3.851035,0.275037,14.033333,8.320000,76.866667,347.533333,0.237187,85.753333,2.842667,36.319824,17.333333
std,2.251440,0.066361,0.670465,1.000857,33.079702,62.956750,0.359496,49.273636,2.271247,16.847317,5.702965
min,1.993000,0.178159,13.100000,5.900000,44.000000,262.000000,0.000000,38.400000,0.770000,13.936120,8.000000
25%,2.299064,0.216111,13.500000,8.150000,53.500000,323.500000,0.000000,52.050000,1.185000,23.392650,13.500000
50%,2.808000,0.279000,14.100000,8.200000,64.000000,330.000000,0.000000,68.000000,1.900000,34.311530,17.000000
75%,4.507440,0.323500,14.700000,8.750000,93.000000,338.500000,0.555000,116.500000,3.600000,44.450490,18.500000
max,9.568764,0.393000,14.900000,10.500000,160.000000,498.000000,0.950000,191.000000,8.700000,65.508330,28.000000


Information for:  cube5_cluster2


,PREDICTED_TN,PREDICTED_TP,PREDICTED_TEMP,PREDICTED_DO,PREDICTED_TURB,PREDICTED_COND,PREDICTED_VEL,PREDICTED_SS,PREDICTED_WDP,PREDICTED_CHLcal,PREDICTED_SECCHI
count,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000,6.000000
mean,3.597144,0.176246,14.466667,8.950000,55.500000,472.000000,0.408333,49.235272,4.016667,43.806912,27.000000
std,0.733582,0.006007,0.488535,2.172326,13.780421,44.036349,0.264984,18.916234,2.083667,25.972109,4.560702
min,2.388667,0.169333,13.900000,5.000000,39.000000,417.000000,0.000000,26.570509,1.200000,10.711221,18.000000
25%,3.264633,0.173061,14.050000,8.700000,46.750000,458.750000,0.275000,36.985281,2.650000,20.932986,27.250000
50%,3.750704,0.174520,14.500000,9.250000,52.000000,463.000000,0.445000,48.145563,4.450000,54.534267,28.500000
75%,4.194036,0.178856,14.875000,10.025000,67.750000,476.250000,0.607500,57.550000,4.825000,63.372268,29.750000
max,4.248363,0.186227,15.000000,11.400000,72.000000,551.000000,0.680000,78.600000,7.000000,67.010894,30.000000


Current Simplical Complex:  ['Stratum 3 SUMMER 93-00: ']
Largest shape is:  ['cube3_cluster17']
Number of nodes is:  1


# Condensing 1-simplices
Currently, many one simplices that we have contain information that means one of them is a subset of the other. To resolve this, we replace them with one cluster with all the indices in one node.

This is stored in the function `clean_getShapes(scomplex)` function. Below is a comparison of running the two functions

In [7]:
allComplices = list(jsonData.keys())
print("Standard shape version")
for key in allComplices[0:1]:
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    nodes = scomplex.get('nodes')
    shapes = getShapes(scomplex)
    for shape in shapes:
        indices = []
        for node in shape:
            indices.append(nodes.get(node))
        indices = list(set([item for sublist in indices for item in sublist]))
        print(str(shape) + " : " + str(indices))

print("Clean shape version")
for key in allComplices[0:1]:
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    nodes = scomplex.get('nodes')
    cleanShapes = clean_getShapes(scomplex)
    for shape in cleanShapes:
        indices = []
        for node in shape:
            indices.append(nodes.get(node))
        indices = list(set([item for sublist in indices for item in sublist]))
        print(str(shape) + " : " + str(indices))

Standard shape version
Current Simplical Complex:  ['Stratum 1 SUMMER 93-00: ']
['cube7_cluster1', 'cube6_cluster0', 'cube3_cluster0', 'cube5_cluster0', 'cube4_cluster0', 'cube5_cluster1', 'cube5_cluster5', 'cube7_cluster0', 'cube2_cluster0', 'cube5_cluster11', 'cube1_cluster0', 'cube2_cluster6'] : [0, 1, 2, 3, 4, 5, 11, 12, 13, 14, 15, 17, 18, 19, 21, 22, 24, 25, 26, 27, 29, 33, 34, 35, 37, 39, 40, 41, 42, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 61, 62, 63, 64, 65, 69, 70, 71, 72, 73, 74, 76, 77, 78, 87, 88, 104, 105, 107, 108, 109, 110, 111, 112, 113, 114, 116, 117, 118, 119]
['cube4_cluster5', 'cube5_cluster8'] : [82]
['cube1_cluster2', 'cube2_cluster2'] : [23]
['cube3_cluster1', 'cube2_cluster3'] : [30]
['cube1_cluster8', 'cube2_cluster10'] : [100]
['cube3_cluster2', 'cube2_cluster7'] : [66]
['cube5_cluster2', 'cube4_cluster1'] : [16, 32]
['cube6_cluster6', 'cube7_cluster6'] : [96, 94, 95]
['cube3_cluster4', 'cube4_cluster6'] : [85]
['cube4_cluster4', 'cube5_clu

# Unique Samples
Something Killian was talking about

In [10]:
allComplices = list(jsonData.keys())
for key in allComplices[0:1]: # remove the indices here to get all the strata for all the time periods
    print("Current Simplical Complex: ", key)
    scomplex = jsonData.get(key)
    


Current Simplical Complex:  ['Stratum 1 SUMMER 93-00: ']
{'cube0_cluster0': [28], 'cube0_cluster1': [38, 44], 'cube0_cluster2': [43], 'cube0_cluster3': [103], 'cube0_cluster4': [115], 'cube1_cluster0': [19], 'cube1_cluster1': [20], 'cube1_cluster2': [23], 'cube1_cluster3': [31], 'cube1_cluster4': [36], 'cube1_cluster5': [38, 44], 'cube1_cluster6': [68], 'cube1_cluster7': [75], 'cube1_cluster8': [100], 'cube1_cluster9': [106], 'cube2_cluster0': [19, 24, 26, 29, 50, 54, 55], 'cube2_cluster1': [20], 'cube2_cluster2': [23], 'cube2_cluster3': [30], 'cube2_cluster4': [31], 'cube2_cluster5': [36], 'cube2_cluster6': [60], 'cube2_cluster7': [66], 'cube2_cluster8': [68], 'cube2_cluster9': [75], 'cube2_cluster10': [100], 'cube2_cluster11': [106], 'cube3_cluster0': [22, 24, 25, 27, 29, 33, 34, 35, 37, 39, 41, 50, 54, 57, 60, 73], 'cube3_cluster1': [30], 'cube3_cluster2': [66], 'cube3_cluster3': [79], 'cube3_cluster4': [85], 'cube3_cluster5': [90], 'cube3_cluster6': [91], 'cube4_cluster0': [15, 18,